In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

The source of all images is wikipedia so no worries for copyright

## LSTM - image

![](images/lstm_im.png)

## LSTM - Equations

Let:

$fn$ = Number of features

$hs$ = Number of output nodes (hidden size)

$bs$ = Batch size

Then:
 * Each $W_{something}$ matrix below has the shape $(fn, hs)$;
 * Each $U_{something}$ matrix below has the shape $(hs, hs)$;
 * Each $b_{something}$ matrix below has the shape $(1, hs)$;
 * The $x_t$ matrix below has shape $(bs, fn)$, corresponding to the element of index $t$ of each sequence inf the batch.; and
 * The $h_t$ matrix below has shape $(bs, hs)$, corresponding to hidden state at time $t$ of each sequence inf the batch.

And:

$f_t = \sigma(W_f \ x_t + U_f \ h_{t-1} + b_f)$

$i_t = \sigma(W_i \ x_t + U_i \ h_{t-1} + b_i)$

$o_t = \sigma(W_o \ x_t + U_o \ h_{t-1} + b_o)$

$g_t = \tanh \ (W_g \ x_t + U_g \ h_{t-1} + b_g)$ a.k.a. $\tilde{c}_t$

$c_t = f_t \circ c_{t-1} + i_t \circ g_t$

$h_t = o_t \circ \tanh \ (c_t)$

In [ ]:
# implementation

In [ ]:
import math
class NaiveCustomLSTM(nn.Module):
    def __init__(self, input_sz: int, hidden_sz: int):
        super().__init__()
        self.input_size = input_sz
        self.hidden_size = hidden_sz
        
        #i_t
        self.W_i = nn.Parameter(torch.Tensor(input_sz, hidden_sz))
        self.U_i = nn.Parameter(torch.Tensor(hidden_sz, hidden_sz))
        self.b_i = nn.Parameter(torch.Tensor(hidden_sz))
        
        #f_t
        self.W_f = nn.Parameter(torch.Tensor(input_sz, hidden_sz))
        self.U_f = nn.Parameter(torch.Tensor(hidden_sz, hidden_sz))
        self.b_f = nn.Parameter(torch.Tensor(hidden_sz))
        
        #c_t
        self.W_c = nn.Parameter(torch.Tensor(input_sz, hidden_sz))
        self.U_c = nn.Parameter(torch.Tensor(hidden_sz, hidden_sz))
        self.b_c = nn.Parameter(torch.Tensor(hidden_sz))
        
        #o_t
        self.W_o = nn.Parameter(torch.Tensor(input_sz, hidden_sz))
        self.U_o = nn.Parameter(torch.Tensor(hidden_sz, hidden_sz))
        self.b_o = nn.Parameter(torch.Tensor(hidden_sz))
        
        self.init_weights()
    
    def init_weights(self):
        stdv = 1.0 / math.sqrt(self.hidden_size)
        for weight in self.parameters():
            weight.data.uniform_(-stdv, stdv)
    
    def forward(self,
                x,
                init_states=None):
        
        """
        assumes x.shape represents (batch_size, sequence_size, input_size)
        """
        bs, seq_sz, _ = x.size()
        hidden_seq = []
        
        if init_states is None:
            h_t, c_t = (
                torch.zeros(bs, self.hidden_size).to(x.device),
                torch.zeros(bs, self.hidden_size).to(x.device),
            )
        else:
            h_t, c_t = init_states
            
        for t in range(seq_sz):
            x_t = x[:, t, :]
            
            i_t = torch.sigmoid(x_t @ self.W_i + h_t @ self.U_i + self.b_i)
            f_t = torch.sigmoid(x_t @ self.W_f + h_t @ self.U_f + self.b_f)
            g_t = torch.tanh(x_t @ self.W_c + h_t @ self.U_c + self.b_c)
            o_t = torch.sigmoid(x_t @ self.W_o + h_t @ self.U_o + self.b_o)
            c_t = f_t * c_t + i_t * g_t
            h_t = o_t * torch.tanh(c_t)
            
            hidden_seq.append(h_t.unsqueeze(0))
        
        #reshape hidden_seq p/ retornar
        hidden_seq = torch.cat(hidden_seq, dim=0)
        hidden_seq = hidden_seq.transpose(0, 1).contiguous()
        return hidden_seq, (h_t, c_t)

In [ ]:
layer = NaiveCustomLSTM(4, 5)

t = torch.ones(10, 3, 4) #batch_size, seq_size, input_size

## LSTM - Time for VECTORIZATION

We are going to make it with only 2 matrix multiplications, to make it much FASTER by running parallelized on MKL / Cuda. 

![](images/vectored.jpg)

Let:

$fn$ = Number of features

$hs$ = Number of output nodes (hidden size)

$bs$ = Batch size

Then:
 * The $W$ matrix below has the shape $(fn, 4 \times hs)$;
 * The $U$ matrix below has the shape $(hs, 4 \times hs)$;
 * The $b$ matrix below has the shape $(1, 4 \times hs)$;
 * The $x_t$ matrix below has shape $(bs, fn)$, corresponding to the element of index $t$ of each sequence inf the batch.;
 * The $h_t$ matrix below has shape $(bs, hs)$, corresponding to hidden state at time $t$ of each sequence inf the batch;
 * $M[x, y]$ indicate the element of the matrix M with at the cordinates (x, y);
 * $M[x, :]$ indicate the x$^{th}$ row of the matrix M; 
 * $M[:, y]$ indicate the y$^{th}$ column of the matrix M; and
 * $M[:, y:y_n]$ indicate the submatrix of M that corresponds all of its rows and columns $y$ to $y_n$
 
And

$A_t = W x_t + U h_t + b$ a.k.a. "Concatenated gates matrix" - runs vectorized

$i_t = \sigma(A_t[:, 0:hs])$

$f_t = \sigma(A_t[:, hs: 2\times hs])$

$o_t = \sigma(A_t[:, 2\times hs: 3\times hs])$

$g_t = \tanh (A_t[:, 3\times hs: 4\times hs])$

$c_t = f_t \circ c_{t-1} + i_t \circ g_t$

$h_t = o_t \circ \tanh \ (c_t)$

In [ ]:
import math

class CustomLSTM(nn.Module):
    def __init__(self, input_sz, hidden_sz):
        super().__init__()
        self.input_sz = input_sz
        self.hidden_size = hidden_sz
        self.W = nn.Parameter(torch.Tensor(input_sz, hidden_sz * 4))
        self.U = nn.Parameter(torch.Tensor(hidden_sz, hidden_sz * 4))
        self.bias = nn.Parameter(torch.Tensor(hidden_sz * 4))
        self.init_weights()
                
    def init_weights(self):
        stdv = 1.0 / math.sqrt(self.hidden_size)
        for weight in self.parameters():
            weight.data.uniform_(-stdv, stdv)
         
    def forward(self, x, 
                init_states=None):
        """Assumes x is of shape (batch, sequence, feature)"""
        bs, seq_sz, _ = x.size()
        hidden_seq = []
        if init_states is None:
            h_t, c_t = (torch.zeros(bs, self.hidden_size).to(x.device), 
                        torch.zeros(bs, self.hidden_size).to(x.device))
        else:
            h_t, c_t = init_states
         
        HS = self.hidden_size
        for t in range(seq_sz):
            x_t = x[:, t, :]
            # batch the computations into a single matrix multiplication
            gates = x_t @ self.W + h_t @ self.U + self.bias
            i_t, f_t, g_t, o_t = (
                torch.sigmoid(gates[:, :HS]), # input
                torch.sigmoid(gates[:, HS:HS*2]), # forget
                torch.tanh(gates[:, HS*2:HS*3]),
                torch.sigmoid(gates[:, HS*3:]), # output
            )
            c_t = f_t * c_t + i_t * g_t
            h_t = o_t * torch.tanh(c_t)
            hidden_seq.append(h_t.unsqueeze(0))
        hidden_seq = torch.cat(hidden_seq, dim=0)
        # reshape from shape (sequence, batch, feature) to (batch, sequence, feature)
        hidden_seq = hidden_seq.transpose(0, 1).contiguous()
        return hidden_seq, (h_t, c_t)